In [4]:
import pandas as pd

In [5]:
# The chromium driver is requried to run the selenium tool
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (101.0.4951.64-0ubuntu0.18.04.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [6]:
# Selenium is requried for the web automation
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.26.9-py2.py3-none-any.whl (138 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [7]:
# To scrap the websites the necessary library packages are imported.
import time
from selenium import webdriver
from selenium.webdriver import Chrome 
from selenium.webdriver.common.by import By 
from selenium.webdriver.common.keys import Keys 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [8]:
# The ScrapComments function setup the environment and scrap the comments.
def ScrapComments(path):
  authors = []                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
  try:
    
    options = webdriver.ChromeOptions()
    options.add_argument('-headless')
    options.add_argument('-no-sandbox')
    options.add_argument('-disable-dev-shm-usage')
    
    driver =webdriver.Chrome('chromedriver',chrome_options=options)
    wait = WebDriverWait(driver,15)
    driver.get(path)
    
    # Sets the range of comments from the API extraction.
    for item in range(5):
      wait.until(EC.visibility_of_element_located((By.TAG_NAME,"body"))).send_keys(Keys.END)
      time.sleep(5)
    
    
    for author in wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME,"style-scope ytd-comment-renderer"))):
      authors.append(author.text)
    
  except Exception as e:
    print(e)
  return authors

# Path defines the user input for the YouTube video link.


In [9]:
# To clean the data we are removing the reply comments, followed by number of replies to that comment
import re
import pandas as pd
def create_df_author_comments1(data):
  author_comment = []
  author=[]
  for item in data:
    s = re.sub('\\n[0-9]+ (days|weeks|months|years|week|month|year|hours|hour|day|minutes|minute) ago' ,'',item)
    s = re.sub('\\nREPLY','',s)
    s = re.sub('\.\\n[0-9][\.]*[0-9]*[A-Z]*','',s)
    t = s.split('\n')
    t[0] = re.sub('\(edited\)','',t[0])
    author.append(t[0])
    s = "\n".join(t[1:])
    author_comment.append(s)
  
  # Here we have put author name as row name and their comment in column inside the data frame.  
  df = pd.DataFrame(author_comment,index=author,columns=["Comments"])
  df.to_csv(r'F:\comments.csv')
  return df


In [10]:
# Importing the necessary libraries for the comment analysis and for visualization of data
from textblob import TextBlob
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [11]:
# Gets the subjectivity where the subject views are pinned.
def getSubjectivity(text):
  return TextBlob(text).sentiment.subjectivity

# Gets the polarity (for checking the positive and negative comments)
def getPolarity(text):
  return TextBlob(text).sentiment.polarity


In [12]:
# Computing the analysis by its score is to diffrentiate it from the positive, negative and neutral.
def getAnalysis(score):
  if score < 0 :
    return 'Negative'
  elif score == 0:
    return 'Neutral'
  else:
    return 'Positive'



In [13]:
def val(data):
  # Conversion of analysis labels into percentage for the output.
  pcomments = data[data.Analysis == 'Positive']
  pcomments = pcomments['Comments']
  positive=round((pcomments.shape[0]/data.shape[0])*100, 1)
  print('Positive: ' +str(round((pcomments.shape[0]/data.shape[0])*100, 1))+ '%')

  ncomments = data[data.Analysis == 'Negative']
  ncomments = ncomments['Comments']
  negative=round((ncomments.shape[0]/data.shape[0])*100, 1)
  print('Negative: ' +str(round((ncomments.shape[0]/data.shape[0])*100, 1))+ '%')

  nucomments = data[data.Analysis == 'Neutral']
  nucomments = nucomments['Comments']
  neutral=round((nucomments.shape[0]/data.shape[0])*100, 1)
  print('Neutral: ' +str(round((nucomments.shape[0]/data.shape[0])*100, 1))+ '%')
  return positive,negative,neutral


In [14]:
!pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.3.0 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [15]:
!pip install flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [33]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
cd /content/drive/My Drive/EDP

/content/drive/My Drive/EDP


In [18]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!ngrok authtoken 2AANgy3GzA0e5ax4uh2j9s4sPcH_87Ff4HSw3hGKJENhLMNVx 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [20]:
def cloud(data):
  allWords = ' '.join( [cmts for cmts in data['Comments']])
  wordCloud = WordCloud(width = 500, height = 300, random_state = 21, max_font_size = 119).generate(allWords)

  plt.imshow(wordCloud, interpolation= 'bilinear')
  plt.axis('off')
  plt.show

In [35]:
from flask_ngrok import run_with_ngrok
import numpy as np
from flask import Flask,request, render_template
#import matplotlib.pyplot as plt
import io
from PIL import Image
from flask import Response
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/',methods =['GET', 'POST'])
def home():
    print("hello")
    if request.method == 'POST' and 'name' in request.form:
      name=request.form['name']
      print("hrllo",name)
      #path = "https://www.youtube.com/watch?v=rfscVS0vtbw"
      #display(name)
      global data
      data = ScrapComments(name)
      data= create_df_author_comments1(data)
      data['Subjectivity'] = data['Comments'].apply(getSubjectivity)
      data['Polarity'] = data['Comments'].apply(getPolarity)
      data['Analysis'] = data['Polarity'].apply(getAnalysis)
      positive,negative,neutral=val(data)
      print(data)
      print("Positive ",positive)
      print("Negative ",negative)
      print("Neutral ",neutral)
      data = {'Task' : 'Hours per Day', 'Positve Review' : positive, 'Negative Review' : negative, 'Neutral Review' : neutral}
      return render_template('graph.html',res1=name,res2=positive,res3=negative,res4=neutral,data=data)
    return render_template('homepage_1.html')


app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4d39-35-245-99-156.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [24/Jun/2022 17:36:23] "GET / HTTP/1.1" 200 -


hello


...

hello


127.0.0.1 - - [24/Jun/2022 17:36:29] "GET /static/assets/images/slider-left-dec.jpg HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:29] "GET /static/assets/images/quote-bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:29] "GET /static/assets/images/footer-bg.jpg HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:30] "GET /static/assets/images/slider-right-dec.jpg HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:30] "GET /static/vendor/bootstrap/css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:30] "GET /static/assets/css/fontawesome.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:30] "GET /static/assets/css/templatemo-digimedia-v1.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:30] "GET /static/assets/css/animated.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:30] "GET /static/vendor/jquery/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:30] "GET /static/assets/css/owl.css HTTP/1.1" 200 -
127.0.0.1 - - [24/Jun/2022 17:36:33] "GE

hello
hrllo https://www.youtube.com/watch?v=Zl_dTUsrU9s


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: use options instead of chrome_options
  # This is added back by InteractiveShellApp.init_path()
127.0.0.1 - - [24/Jun/2022 17:37:39] "POST / HTTP/1.1" 200 -


Positive: 78.8%
Negative: 7.5%
Neutral: 13.8%
                                                                Comments  \
Agent 47 Gaming India  This has to be the most anticipated and best s...   
Kennethmarco1          The Song "Running Up that hill" Was So Insane ...   
Nikki                  This has got to be one of the best shows ever ...   
Mukesh Srivastav       One of the key reasons for this season's succe...   
Mike Scott             This version of “Running Up That Hill” where i...   
...                                                                  ...   
Boomer                 this must be the most awesome season for life!...   
MxMo                   Everything is amazing in this trailer, the mus...   
nishit shah            The scene where Eddie plays the guitar in upsi...   
Rosie posie            Eddie with that guitar is going to be one of t...   
eduardo heggendorn     This is in another level for sure ! Can’t desc...   

                       Subjectivity  Pola